In [4]:
import time
import json
import nest_asyncio
import asyncio
import websockets
import numpy as np
import math
import random
import colorsys
from tqdm.notebook import tqdm
nest_asyncio.apply()

In [5]:
async def websocket_test(payload, indent=None, quiet=False):
    uri = "ws://127.0.0.1:8765/ws"
    async with websockets.connect(uri) as ws:
        await ws.send(json.dumps(payload))
        if not quiet:
            print("✅ send:", payload)

        try:
            # 5 秒以内に返事が来なければ TimeoutError
            response = await asyncio.wait_for(ws.recv(), timeout=5)
            if not quiet:
                if indent:
                    print("📨 recv: ", json.dumps(json.loads(response), indent=indent, ensure_ascii=False))
                else:
                    print("📨 recv:", response)
            return json.loads(response)
        except asyncio.TimeoutError:
            print("⏰ タイムアウト（5 秒以内にメッセージが来ませんでした）")

In [21]:

size = 0.05

for i in tqdm(range(100)):

    # 平面の追加
    payload = {
        "type": "add",
        "command": "item",
        "group": 0,
        "category": 0,
        "itemId": 740,
    }
    item_response = await websocket_test(payload, quiet=True)

    # フォルダの追加
    payload = {
        "type": "add",
        "command": "folder",
        "name": "星"
    }
    star_response = await websocket_test(payload, quiet=True)

    # 星空 -> 星
    payload = {
        "type": "hierarchy",
        "command": "attach",
        "parentId": 131,
        "childId": star_response["objectId"],
    }
    response = await websocket_test(payload, quiet=True)

    # 星 -> 星オブジェクト
    payload = {
        "type": "hierarchy",
        "command": "attach",
        "parentId": star_response["objectId"],
        "childId": item_response["objectId"],
    }
    response = await websocket_test(payload, quiet=True)

    # 星オブジェクトの変形
    payload = {
        "type": "update",
        "command": "transform",
        "id": item_response["objectId"],
        "pos": [-20.0, 0.0, 0.0],
        "rot": [0.0, 0.0, 270.0],
        "scale": [size, 0.01, size],
    }
    response = await websocket_test(payload, quiet=True)

    # 星の回転
    payload = {
        "type": "update",
        "command": "transform",
        "id": star_response["objectId"],
        "pos": [0.0, 0.0, 0.0],
        "rot": [
            0.0,
            random.uniform(-17, 17),
            random.uniform(-10, 0),
        ],
    }
    response = await websocket_test(payload, quiet=True)


  0%|          | 0/100 [00:00<?, ?it/s]